In [1]:
import sys
import os
import google.generativeai as genai
import speech_recognition as sr
import pyttsx3
import webbrowser
import subprocess
from PyQt5.QtWidgets import QApplication, QMainWindow, QWidget, QVBoxLayout, \
    QHBoxLayout, QPushButton, QLineEdit, QTextEdit, QLabel
from PyQt5.QtGui import QFont

class ChatWindow(QMainWindow):
    def __init__(self):
        super().__init__()
        self.init_ui()

    def init_ui(self):
        os.environ['GOOGLE_API_KEY'] = "put your api key here"
        genai.configure(api_key=os.environ['GOOGLE_API_KEY'])
        self.model = genai.GenerativeModel('gemini-pro')

        self.setWindowTitle("Desktop Assistant")  # Set window title
        self.setGeometry(100, 100, 600, 400)

        central_widget = QWidget()
        self.setCentralWidget(central_widget)

        layout = QVBoxLayout()
        central_widget.setLayout(layout)

        # Input box
        input_layout = QHBoxLayout()
        layout.addLayout(input_layout)

        input_label = QLabel("Input:")
        input_layout.addWidget(input_label)

        self.user_input = QLineEdit()
        self.user_input.setStyleSheet("background-color: black; color: white;")  
        input_layout.addWidget(self.user_input)

        
        self.chat_history_display = QTextEdit()
        self.chat_history_display.setStyleSheet("background-color: black; color: white;") 
        self.chat_history_display.setReadOnly(True)  
        layout.addWidget(self.chat_history_display)

        
        self.chat_history_display.append("How can I help?")

        # Send button
        send_button = QPushButton("Send")
        layout.addWidget(send_button)
        send_button.clicked.connect(self.send_message)

        # Voice input button
        voice_button = QPushButton("Voice Input")
        layout.addWidget(voice_button)
        voice_button.clicked.connect(self.listen_voice_input)

        self.setStyleSheet("background-color: white;")  # Set main window background color to white
        self.show()

        
        self.tts_engine = pyttsx3.init()

    def send_message(self):
        query = self.user_input.text().strip()
        self.user_input.clear()

        if query.lower() == "quit":
            self.close()
            return

        if query.lower().startswith("search"):
            search_query = query[len("search"):].strip()
            self.update_chat_history(f"Searching for: {search_query}")
            webbrowser.open(f"https://www.google.com/search?q={search_query}")
            return

        if query.lower().startswith("open"):
            app_name = query[len("open"):].strip()
            if app_name.lower() == "camera":
                try:
                    subprocess.Popen(["start", "cmd", "/k", "start", "microsoft.windows.camera:"], shell=True)
                    self.update_chat_history(f"Opening {app_name}")
                except Exception as e:
                    self.update_chat_history(f"Error: {e}")
            else:
                self.update_chat_history("Unsupported application")
            return

        response = self.model.generate_content(query)
        response_text = response.text.replace('*', '')  
        self.update_chat_history(f"Assistant: {response_text}\n")
        self.read_out_loud(response_text)  

    def listen_voice_input(self):
        recognizer = sr.Recognizer()
        with sr.Microphone() as source:
            print("Listening...")
            recognizer.adjust_for_ambient_noise(source)
            audio = recognizer.listen(source)

        try:
            query = recognizer.recognize_google(audio)
            self.user_input.setText(query)
            self.send_message()
        except sr.UnknownValueError:
            print("Could not understand audio")
        except sr.RequestError as e:
            print(f"Could not request results; {e}")

    def update_chat_history(self, text):
        self.chat_history_display.append(text)  # Append the text to the response box
        self.chat_history_display.verticalScrollBar().setValue(self.chat_history_display.verticalScrollBar().maximum())

    def read_out_loud(self, text):
        self.tts_engine.say(text)
        self.tts_engine.runAndWait()

if __name__ == '__main__':
    app = QApplication(sys.argv)
    window = ChatWindow()
    sys.exit(app.exec_())


Listening...
Could not understand audio


SystemExit: 0

C:\Users\jaden\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3468: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
